In [1]:
# Parameters
alpha = 0.6
ratio = 0.1


In [2]:
# 1. Importaciones necesarias
import pandas as pd
import glob
import os

# 2. Definición de variables
season_ids = ['4024', '4232', '4019', '4104']  # Ahora como strings
temporada = '24/25'
input_folder = '../../data_backup/archivos_parquet'
output_folder = '../../data/archivos_parquet'


In [3]:
# 3. Crear carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Carpeta de salida creada: {output_folder}")

In [4]:
# 4. Procesar archivos
print("Iniciando procesamiento de archivos...")
print("-" * 50)

# Obtener lista de archivos parquet
all_files = glob.glob(os.path.join(input_folder, '*all*.parquet'))
eventos_files = glob.glob(os.path.join(input_folder, '*eventos*.parquet'))
kpi_files = glob.glob(os.path.join(input_folder, '*KPI*.parquet'))  # Añadir búsqueda de archivos KPI
datos_files = glob.glob(os.path.join(input_folder, '*datos*.parquet'))  # Añadir búsqueda de archivos datos
parquet_files = list(set(all_files + eventos_files + kpi_files + datos_files))  # Incluir datos_files

print(f"Total de archivos a procesar: {len(parquet_files)}")

# Procesar cada archivo
for file_path in parquet_files:
    try:
        # Leer el archivo parquet
        df = pd.read_parquet(file_path)
        file_name = os.path.basename(file_path)
        
        print(f"Procesando: {file_name}")
        
        # Verificar si existe la columna season_id y/o temporada
        has_season_id = 'season_id' in df.columns
        has_temporada = 'temporada' in df.columns
        
        if has_season_id or has_temporada:
            # Inicializar df_filtered con todos los datos
            df_filtered = df
            
            # Aplicar filtros según las columnas disponibles
            if has_season_id:
                # Convertir season_id a string para la comparación
                df_filtered['season_id'] = df_filtered['season_id'].astype(str)
                df_filtered = df_filtered[df_filtered['season_id'].isin(season_ids)]
                print("✓ Filtrado por season_id aplicado")
                print(f"Valores únicos de season_id encontrados: {df_filtered['season_id'].unique()}")
            
            if has_temporada:
                df_filtered = df_filtered[df_filtered['temporada'] == temporada]
                print("✓ Filtrado por temporada aplicado")
                print(f"Valores únicos de temporada encontrados: {df_filtered['temporada'].unique()}")
            
            # Si hay datos después del filtrado
            if not df_filtered.empty:
                # Crear la ruta de salida
                output_path = os.path.join(output_folder, file_name)
                
                try:
                    # Intentar guardar el archivo
                    df_filtered.to_parquet(output_path, engine='pyarrow', index=False)
                    
                    # Verificar que el archivo se creó
                    if os.path.exists(output_path):
                        print(f"✓ Archivo guardado exitosamente en: {output_path}")
                        print(f"✓ Tamaño del archivo: {os.path.getsize(output_path)} bytes")
                    else:
                        print(f"✗ Error: El archivo no se guardó en {output_path}")
                    
                    print(f"✓ Registros originales: {len(df)}")
                    print(f"✓ Registros después del filtrado: {len(df_filtered)}")
                
                except Exception as e:
                    print(f"✗ Error al guardar el archivo: {str(e)}")
                    # Verificar permisos de la carpeta
                    print(f"Verificando permisos de la carpeta destino:")
                    print(f"¿Existe la carpeta?: {os.path.exists(output_folder)}")
                    print(f"¿Se puede escribir en la carpeta?: {os.access(output_folder, os.W_OK)}")
        else:
            print("✗ El archivo no contiene ni season_id ni temporada")
            print(f"Columnas disponibles: {df.columns.tolist()}")
    
    except Exception as e:
        print(f"✗ Error procesando el archivo: {str(e)}")
        print("-" * 50)

print("Procesamiento completado!")


Iniciando procesamiento de archivos...
--------------------------------------------------
Total de archivos a procesar: 16
Procesando: sequence_data_league_all.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/sequence_data_league_all.parquet
✓ Tamaño del archivo: 3004860 bytes
✓ Registros originales: 263349
✓ Registros después del filtrado: 98096
Procesando: KPI_equipos.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4019' '4024' '4104' '4232']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/KPI_equipos.parquet
✓ Tamaño del archivo: 57087 bytes
✓ Registros originales: 924
✓ Registros después del filtrado: 328
Procesando: formations_league_all.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']
✓ Archivo guardado exitosamente en: ../../data/archivos

✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/players_league_all.parquet
✓ Tamaño del archivo: 39973 bytes
✓ Registros originales: 9642
✓ Registros después del filtrado: 5600
Procesando: lineups_league_all.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/lineups_league_all.parquet
✓ Tamaño del archivo: 116676 bytes
✓ Registros originales: 16793
✓ Registros después del filtrado: 6230
Procesando: eventos_datos_acumulados.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4019' '4024' '4104' '4232']
✓ Filtrado por temporada aplicado
Valores únicos de temporada encontrados: ['24/25']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/eventos_datos_acumulados.parquet
✓ Tamaño del archivo: 924084 by

Procesando: eventos_metricas_alaves.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']


✓ Filtrado por temporada aplicado
Valores únicos de temporada encontrados: ['24/25']


✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/eventos_metricas_alaves.parquet
✓ Tamaño del archivo: 19817791 bytes
✓ Registros originales: 810068
✓ Registros después del filtrado: 274305
Procesando: leagues_all.parquet
✗ El archivo no contiene ni season_id ni temporada
Columnas disponibles: ['id', 'name', 'name_en', 'iso_country_code', 'age_limit', 'division', 'season_ids']


Procesando: events_league_all.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']


✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/events_league_all.parquet
✓ Tamaño del archivo: 12820515 bytes
✓ Registros originales: 904372
✓ Registros después del filtrado: 313058
Procesando: player_stats_extended_league_all.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4024' '4232' '4019' '4104']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/player_stats_extended_league_all.parquet
✓ Tamaño del archivo: 325587 bytes
✓ Registros originales: 14532
✓ Registros después del filtrado: 5375
Procesando: df_eventos_acumulados_sin_KPI.parquet
✓ Filtrado por season_id aplicado
Valores únicos de season_id encontrados: ['4019' '4024' '4104' '4232']
✓ Filtrado por temporada aplicado
Valores únicos de temporada encontrados: ['24/25']
✓ Archivo guardado exitosamente en: ../../data/archivos_parquet/df_eventos_acumulados_sin_KPI.parquet
✓ Tamaño del archivo: 766964 bytes
✓ Registros originales: 14650
✓ Registros después del fil

In [5]:
# 5. Verificación de resultados
print("\nINICIANDO VERIFICACIÓN")
print("=" * 50)

# Verificar archivos en ambas carpetas
input_files = glob.glob(os.path.join(input_folder, '*all*.parquet')) 
output_files = glob.glob(os.path.join(output_folder, '*.parquet'))

print(f"Total archivos de entrada: {len(input_files)}")
print(f"Total archivos de salida: {len(output_files)}")

# Verificar contenido de los archivos procesados
print("\nVerificación de archivos procesados:")
print("-" * 50)

for output_file in output_files:
    try:
        df = pd.read_parquet(output_file)
        file_name = os.path.basename(output_file)
        
        print(f"\nArchivo: {file_name}")
        
        # Verificar columnas y valores
        if 'season_id' in df.columns:
            unique_seasons = df['season_id'].unique()
            print(f"Season IDs encontrados: {sorted(unique_seasons)}")
            is_correct_season = all(sid in season_ids for sid in unique_seasons)
            print(f"¿Todos los season_ids son correctos?: {'✓' if is_correct_season else '✗'}")
        
        if 'temporada' in df.columns:
            unique_temporadas = df['temporada'].unique()
            print(f"Temporadas encontradas: {sorted(unique_temporadas)}")
            is_correct_temp = all(temp == temporada for temp in unique_temporadas)
            print(f"¿Todas las temporadas son correctas?: {'✓' if is_correct_temp else '✗'}")
        
        print(f"Total de registros: {len(df)}")
        
        # Mostrar distribución de datos
        print("\nDistribución de datos:")
        if 'season_id' in df.columns:
            print("Por season_id:")
            distribution = df['season_id'].value_counts().sort_index()
            for sid, count in distribution.items():
                print(f"season_id {sid}: {count} registros")
        
        if 'temporada' in df.columns:
            print("\nPor temporada:")
            distribution = df['temporada'].value_counts().sort_index()
            for temp, count in distribution.items():
                print(f"temporada {temp}: {count} registros")
            
    except Exception as e:
        print(f"Error en la verificación: {str(e)}")
    
    print("-" * 50)

print("\nProceso de verificación completado!")


INICIANDO VERIFICACIÓN
Total archivos de entrada: 12
Total archivos de salida: 15

Verificación de archivos procesados:
--------------------------------------------------

Archivo: eventos_datos_acumulados.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Temporadas encontradas: ['24/25']
¿Todas las temporadas son correctas?: ✓
Total de registros: 5023

Distribución de datos:
Por season_id:
season_id 4019: 2310 registros
season_id 4024: 1337 registros
season_id 4104: 742 registros
season_id 4232: 634 registros

Por temporada:
temporada 24/25: 5023 registros
--------------------------------------------------

Archivo: seasons_teams_all.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Total de registros: 28

Distribución de datos:
Por season_id:
season_id 4019: 7 registros
season_id 4024: 7 registros
season_id 4104: 7 registros
season_id 4232: 7 registros
----------------------


Archivo: eventos_metricas_alaves.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Temporadas encontradas: ['24/25']
¿Todas las temporadas son correctas?: ✓
Total de registros: 274305

Distribución de datos:
Por season_id:
season_id 4019: 127784 registros
season_id 4024: 76656 registros
season_id 4104: 38164 registros
season_id 4232: 31701 registros

Por temporada:
temporada 24/25: 274305 registros
--------------------------------------------------

Archivo: names_players_league_all.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Total de registros: 5600

Distribución de datos:
Por season_id:
season_id 4019: 1303 registros
season_id 4024: 1677 registros
season_id 4104: 1377 registros
season_id 4232: 1243 registros
--------------------------------------------------

Archivo: df_eventos_acumulados_sin_KPI.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos 


Archivo: matches_league_all.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Total de registros: 272

Distribución de datos:
Por season_id:
season_id 4019: 97 registros
season_id 4024: 73 registros
season_id 4104: 53 registros
season_id 4232: 49 registros
--------------------------------------------------

Archivo: lineups_league_all.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Total de registros: 6230

Distribución de datos:
Por season_id:
season_id 4019: 2836 registros
season_id 4024: 1612 registros
season_id 4104: 967 registros
season_id 4232: 815 registros
--------------------------------------------------

Archivo: player_stats_extended_league_all.parquet
Season IDs encontrados: ['4019', '4024', '4104', '4232']
¿Todos los season_ids son correctos?: ✓
Total de registros: 5375

Distribución de datos:
Por season_id:
season_id 4019: 2410 registros
season_id 4024: 1398 r